In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wrangle
import model


from importlib import reload
from itertools import product
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [67]:
train, X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.model_prep()

In [68]:
# formulate baseline prediction
train.employed.value_counts(normalize=True)

1    0.968006
0    0.031994
Name: employed, dtype: float64

In [69]:
train

,household_num,family_income,own_bus_or_farm,metro_area_size,age,is_male,veteran,hispanic_or_non,employed,enrolled_in_school,...,"occupation_Management, business, and financial occupations",occupation_Office and administrative support occupations,occupation_Production occupations,occupation_Professional and related occupations,occupation_Sales and related occupations,occupation_Service occupations,occupation_Transportation and material moving occupations,service_era_n/a,service_era_post_gwot,service_era_pre_gwot
3549,0.000000,14.0,0,5.0,37.0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,0,0
33977,0.083333,16.0,0,6.0,35.0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
28289,0.166667,14.0,1,7.0,43.0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
7071,0.333333,15.0,0,7.0,58.0,1,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
25033,0.166667,7.0,0,3.0,39.0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,0.083333,15.0,1,3.0,46.0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
35627,0.250000,11.0,0,0.0,21.0,0,0,0,1,1,...,0,0,0,0,0,1,0,1,0,0
9382,0.166667,16.0,0,7.0,45.0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
20950,0.083333,12.0,0,4.0,27.0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0


# GridsearchCV

## Decision Tree Classifer

In [70]:
params = {'max_depth': [10,11,12,13,14,15],
          'max_features': [10,20,25,30,35,40]}

dtc = DecisionTreeClassifier()

grid = GridSearchCV(dtc, params, cv=3, scoring='precision')

grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15],
                         'max_features': [10, 20, 25, 30, 35, 40]},
             scoring='precision')

In [71]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_max_features', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [72]:
test_scores = results['mean_test_score']

In [73]:
params = results['params']

In [74]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,max_depth,max_features,score
18,13,10,0.968419
0,10,10,0.968501
6,11,10,0.968573
24,14,10,0.968601
8,11,25,0.968647
5,10,40,0.968663
13,12,20,0.968821
7,11,20,0.968846
30,15,10,0.968945
20,13,25,0.968951


## K Neighbors

In [75]:
params = {'n_neighbors': [5,6,7,8],
          'weights': ('uniform', 'distance')}

knn = KNeighborsClassifier()

grid = GridSearchCV(knn, params, cv=3, scoring='precision')

grid.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 6, 7, 8],
                         'weights': ('uniform', 'distance')},
             scoring='precision')

In [76]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'param_weights', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [77]:
test_scores = results['mean_test_score']

In [78]:
params = results['params']

In [79]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score', ascending=False)

,n_neighbors,weights,score
2,6,uniform,0.968493
1,5,distance,0.968490
3,6,distance,0.968382
0,5,uniform,0.968318
5,7,distance,0.968269
6,8,uniform,0.968261
7,8,distance,0.968237
4,7,uniform,0.968167


## Logistic Regression

In [80]:
params = {'solver': ('sag', 'saga'),
          'penalty': ['l2'],
          'C': [.1,.5,1.0]}

logr = LogisticRegression()

grid = GridSearchCV(logr, params, cv=3, scoring='precision')

grid.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linea

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 0.5, 1.0], 'penalty': ['l2'],
                         'solver': ('sag', 'saga')},
             scoring='precision')

In [81]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_penalty', 'param_solver', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [82]:
test_scores = results['mean_test_score']

In [83]:
params = results['params']

In [84]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,C,penalty,solver,score
0,0.1,l2,sag,0.968041
1,0.1,l2,saga,0.968041
3,0.5,l2,saga,0.968041
5,1.0,l2,saga,0.968041
2,0.5,l2,sag,0.968076
4,1.0,l2,sag,0.968111


In [85]:
from sklearn.svm import LinearSVC

## Linear SVC

In [86]:
params = {'dual':(True, False),
          'C': [1e-5, 1e-4, 1e-3,1,10],
          'max_iter': (500,1000)}

lsvc = LinearSVC()

grid = GridSearchCV(lsvc, params, cv=3, scoring='accuracy')

grid.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarni

GridSearchCV(cv=3, estimator=LinearSVC(),
             param_grid={'C': [1e-05, 0.0001, 0.001, 1, 10],
                         'dual': (True, False), 'max_iter': (500, 1000)},
             scoring='accuracy')

In [87]:
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_dual', 'param_max_iter', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [88]:
test_scores = results['mean_test_score']

In [89]:
params = results['params']

In [90]:
for p,s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,C,dual,max_iter,score
0,0.00001,True,500,0.968006
17,10.00000,True,1000,0.968006
16,10.00000,True,500,0.968006
15,1.00000,False,1000,0.968006
14,1.00000,False,500,0.968006
13,1.00000,True,1000,0.968006
12,1.00000,True,500,0.968006
11,0.00100,False,1000,0.968006
10,0.00100,False,500,0.968006
9,0.00100,True,1000,0.968006


## Takeaways
* Decision Tree Classifier best performing type of model. Will roll with that through remaining portion of model section



# Modeling with DTC

In [91]:
subsets = [train, X_train, y_train, X_validate, y_validate, X_test, y_test]

In [92]:
feat_set = X_train.columns

In [93]:
def model_dtc(feat_set,\
        model_descriptions,
        comparison_chart,
        subsets):
    
    train=subsets[0]
    X_train=subsets[1]
    y_train=subsets[2]

    features = []
    for feature in feat_set:
        features += [col for col in train.columns if feature in col]

    depths = list(np.arange(20,31,1))
    min_sample_splits = list(np.arange(10,21,1))
    max_features_set = [75]
    selectors = list(product(depths, min_sample_splits, max_features_set))

    for idx, item in enumerate(selectors):
        model_id = 'DTC_'+f'{idx}'
        dtc = DecisionTreeClassifier(max_depth=item[0], min_samples_split=item[1], max_features=item[2],\
                                            random_state=514)
        
        dtc.fit(X_train[features], y_train)

        comparison_chart[model_id] = model.compute_metrics(dtc, X_train[features], y_train).values

        score = dtc.score(X_train[features], y_train).round(4)

        description = pd.DataFrame({'Model': model_id,
                                    'Accuracy(Score)': score,
                                    'Type': 'Decision Tree Classifier',
                                    'Features Used': f'Implicit',
                                    'Parameters': f'Depth: {item[0]},M.Sample Split: {item[1]}, M.Feat: {item[2]}'},
                                    index=[0])

        model_descriptions = pd.concat([model_descriptions, description], ignore_index=True)
        comparison_chart = comparison_chart.copy() #to avoid fragmentation

    return model_descriptions, comparison_chart

In [94]:
comparison_chart = model.create_comp_chart()

In [95]:
model_descriptions = model.create_description_chart(y_train)

In [96]:
model_descriptions, comparison_chart =  model_dtc(feat_set, model_descriptions, comparison_chart, subsets)

In [97]:
comparison_chart.T.sort_values(by='True Negative Rate', ascending=False).head(10)

,Accuracy/Score,True Positives,False Positives,True Negatives,False Negatives,TPR/Recall,False Positive Rate,True Negative Rate,False Negative Rate,Precision,F1-Score,Support Positive,Support Negative
DTC_99,0.9784,26690.0,362.0,528.0,238.0,0.9912,0.4067,0.5933,0.0088,0.9866,0.9889,26928.0,890.0
DTC_110,0.9788,26711.0,373.0,517.0,217.0,0.9919,0.4191,0.5809,0.0081,0.9862,0.9891,26928.0,890.0
DTC_77,0.9782,26704.0,383.0,507.0,224.0,0.9917,0.4303,0.5697,0.0083,0.9859,0.9888,26928.0,890.0
DTC_45,0.9785,26727.0,397.0,493.0,201.0,0.9925,0.4461,0.5539,0.0075,0.9854,0.9889,26928.0,890.0
DTC_88,0.9775,26703.0,402.0,488.0,225.0,0.9916,0.4517,0.5483,0.0084,0.9852,0.9884,26928.0,890.0
DTC_100,0.9773,26702.0,405.0,485.0,226.0,0.9916,0.4551,0.5449,0.0084,0.9851,0.9883,26928.0,890.0
DTC_33,0.9776,26713.0,408.0,482.0,215.0,0.9920,0.4584,0.5416,0.0080,0.9850,0.9885,26928.0,890.0
DTC_90,0.9772,26703.0,410.0,480.0,225.0,0.9916,0.4607,0.5393,0.0084,0.9849,0.9882,26928.0,890.0
DTC_111,0.9781,26729.0,410.0,480.0,199.0,0.9926,0.4607,0.5393,0.0074,0.9849,0.9887,26928.0,890.0
DTC_66,0.9780,26727.0,411.0,479.0,201.0,0.9925,0.4618,0.5382,0.0075,0.9849,0.9887,26928.0,890.0


In [98]:
testy = comparison_chart.T.sort_values(by='True Negative Rate', ascending=False).head(10).index.values.tolist()

In [99]:
top_10 = model_descriptions[model_descriptions.Model.isin(testy)]

In [100]:
model_descriptions.sort_values(by='Accuracy(Score)', ascending=False).head(10)

,Model,Accuracy(Score),Type,Features Used,Parameters
111,DTC_110,0.9788,Decision Tree Classifier,Implicit,"Depth: 30,M.Sample Split: 10, M.Feat: 75"
46,DTC_45,0.9785,Decision Tree Classifier,Implicit,"Depth: 24,M.Sample Split: 11, M.Feat: 75"
100,DTC_99,0.9784,Decision Tree Classifier,Implicit,"Depth: 29,M.Sample Split: 10, M.Feat: 75"
78,DTC_77,0.9782,Decision Tree Classifier,Implicit,"Depth: 27,M.Sample Split: 10, M.Feat: 75"
112,DTC_111,0.9781,Decision Tree Classifier,Implicit,"Depth: 30,M.Sample Split: 11, M.Feat: 75"
67,DTC_66,0.9780,Decision Tree Classifier,Implicit,"Depth: 26,M.Sample Split: 10, M.Feat: 75"
79,DTC_78,0.9779,Decision Tree Classifier,Implicit,"Depth: 27,M.Sample Split: 11, M.Feat: 75"
12,DTC_11,0.9777,Decision Tree Classifier,Implicit,"Depth: 21,M.Sample Split: 10, M.Feat: 75"
45,DTC_44,0.9776,Decision Tree Classifier,Implicit,"Depth: 24,M.Sample Split: 10, M.Feat: 75"
34,DTC_33,0.9776,Decision Tree Classifier,Implicit,"Depth: 23,M.Sample Split: 10, M.Feat: 75"


## Takeaways 
* Round 1
    * Better True Negative Rate performance with higher Depth and lower sample splits. 
    * Max Features between 25 and above performed well
* Round 2
    * Min Sample Split of 10 is consitent high performer. Will go for 10,11,12 as locked in options
    * Higher Max features is helping to drive up True Negative rate will push 50+ then lock in
    * Depth aabove 20 is superior will push 25 plus then lock in
* Round 3
    * added None as max feature option and it outperformed all Min sample split still best at 10
    * Depth still with room for upwards growth
    .9799 Accuracy and 61% TNR with Depth: 30 ,M.Sample Split: 10, M.Feat: None
* Last Round 
    * Lowered Sample Split down to 5 and got better performance
    * Max Features
    * Depth 31 was the best performing with TNR of 87.3% 
    * Will take Top 10 to validate
* Revisiting
    * Use of None for max features is causing overfitting. will switch to range 40~50
* Final Round 
    * Top 10 

In [101]:
for item in top_10.index:
    print(top_10.loc[item]['Parameters'])

Depth: 23,M.Sample Split: 10, M.Feat: 75
Depth: 24,M.Sample Split: 11, M.Feat: 75
Depth: 26,M.Sample Split: 10, M.Feat: 75
Depth: 27,M.Sample Split: 10, M.Feat: 75
Depth: 28,M.Sample Split: 10, M.Feat: 75
Depth: 28,M.Sample Split: 12, M.Feat: 75
Depth: 29,M.Sample Split: 10, M.Feat: 75
Depth: 29,M.Sample Split: 11, M.Feat: 75
Depth: 30,M.Sample Split: 10, M.Feat: 75
Depth: 30,M.Sample Split: 11, M.Feat: 75


In [102]:
pesty = top_10.Parameters.values[0]

In [103]:
top_10.Parameters.values[0]

'Depth: 23,M.Sample Split: 10, M.Feat: 75'

In [104]:
import re

In [105]:
lesty = re.findall(r'\d\d', pesty)

In [115]:
def validate_dtc(feat_set,\
        top_10,
        subsets):
    
    train=subsets[0]
    X_train=subsets[1]
    y_train=subsets[2]
    X_validate=subsets[3]
    y_validate=subsets[4]

    features = []
    for feature in feat_set:
        features += [col for col in train.columns if feature in col]
        
    depths = list(np.arange(20,35,1))
    min_sample_splits = list(np.arange(5,13,1))
    max_features_set = [55, 75, None]
    selectors = list(product(depths, min_sample_splits, max_features_set))


    validate_descriptions = model.create_description_chart(y_train)
    val_comparisons = model.create_comp_chart()

    for item in top_10.index:
        model_id = top_10.loc[item]['Model']
        selectors = re.findall(r'\d\d', top_10.loc[item]['Parameters'])

        dtc = DecisionTreeClassifier(max_depth=int(selectors[0]), min_samples_split=int(selectors[1]), max_features=int(selectors[2]),\
                                            random_state=514)
        
        dtc.fit(X_train[features], y_train)

        val_comparisons[model_id] = model.compute_metrics(dtc, X_validate[features], y_validate).values

        score = dtc.score(X_validate[features], y_validate).round(4)

        description = pd.DataFrame({'Model': model_id,
                                    'Accuracy(Score)': score,
                                    'Type': 'Decision Tree Classifier',
                                    'Features Used': f'{feat_set}',
                                    'Parameters': f'Depth: {selectors[0]}, M.Sample Split: {selectors[1]}, M.Feat: {selectors[2]}'},
                                    index=[0])

        validate_descriptions = pd.concat([validate_descriptions, description], ignore_index=True)
        val_comparisons = val_comparisons.copy()

    return validate_descriptions, val_comparisons

In [116]:
validate_descriptions, val_comparisons =  validate_dtc(feat_set, top_10, subsets)

In [117]:
val_comparisons.T.sort_values(by='True Negative Rate', ascending=False)

,Accuracy/Score,True Positives,False Positives,True Negatives,False Negatives,TPR/Recall,False Positive Rate,True Negative Rate,False Negative Rate,Precision,F1-Score,Support Positive,Support Negative
DTC_90,0.9457,11233.0,339.0,42.0,308.0,0.9733,0.8898,0.1102,0.0267,0.9707,0.9720,11541.0,381.0
DTC_45,0.9497,11281.0,340.0,41.0,260.0,0.9775,0.8924,0.1076,0.0225,0.9707,0.9741,11541.0,381.0
DTC_111,0.9483,11267.0,342.0,39.0,274.0,0.9763,0.8976,0.1024,0.0237,0.9705,0.9734,11541.0,381.0
DTC_77,0.9454,11233.0,343.0,38.0,308.0,0.9733,0.9003,0.0997,0.0267,0.9704,0.9718,11541.0,381.0
DTC_88,0.9463,11244.0,343.0,38.0,297.0,0.9743,0.9003,0.0997,0.0257,0.9704,0.9723,11541.0,381.0
DTC_99,0.9447,11225.0,343.0,38.0,316.0,0.9726,0.9003,0.0997,0.0274,0.9703,0.9715,11541.0,381.0
DTC_100,0.9435,11210.0,343.0,38.0,331.0,0.9713,0.9003,0.0997,0.0287,0.9703,0.9708,11541.0,381.0
DTC_33,0.9459,11240.0,344.0,37.0,301.0,0.9739,0.9029,0.0971,0.0261,0.9703,0.9721,11541.0,381.0
DTC_66,0.9469,11252.0,344.0,37.0,289.0,0.9750,0.9029,0.0971,0.0250,0.9703,0.9726,11541.0,381.0
DTC_110,0.9483,11269.0,344.0,37.0,272.0,0.9764,0.9029,0.0971,0.0236,0.9704,0.9734,11541.0,381.0


## Validate Takeaways
* Round 1
    * Huge performance loss, appears model was over fit. Will adjust 
* Round 2 
    * Pulled in more models but unable to get above 13% TNR. Will redjust starting from Train section
* Round 3
    * Lower depth performing better with larger min sample splits. Need to increase generalisability
* Final Round
    * Unable to crack 12% with adjustments. Will follow on to model in order to finish MVP and revisit different model types 

In [118]:
validate_descriptions.sort_values(by='Accuracy(Score)', ascending=False)

,Model,Accuracy(Score),Type,Features Used,Parameters
2,DTC_45,0.9497,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 24, M.Sample Split: 11, M.Feat: 75"
9,DTC_110,0.9483,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 30, M.Sample Split: 10, M.Feat: 75"
10,DTC_111,0.9483,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 30, M.Sample Split: 11, M.Feat: 75"
3,DTC_66,0.9469,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 26, M.Sample Split: 10, M.Feat: 75"
5,DTC_88,0.9463,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 28, M.Sample Split: 10, M.Feat: 75"
1,DTC_33,0.9459,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 23, M.Sample Split: 10, M.Feat: 75"
6,DTC_90,0.9457,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 28, M.Sample Split: 12, M.Feat: 75"
4,DTC_77,0.9454,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 27, M.Sample Split: 10, M.Feat: 75"
7,DTC_99,0.9447,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 29, M.Sample Split: 10, M.Feat: 75"
8,DTC_100,0.9435,Decision Tree Classifier,"Index(['household_num', 'family_income', 'own_...","Depth: 29, M.Sample Split: 11, M.Feat: 75"


In [120]:
top_1 = validate_descriptions.iloc[6]

## Test Subset

In [135]:
def test_dtc(feat_set,\
        top_1,
        subsets):
    
    train=subsets[0]
    X_train=subsets[1]
    y_train=subsets[2]
    X_test=subsets[5]
    y_test=subsets[6]

    features = []
    for feature in feat_set:
        features += [col for col in train.columns if feature in col]

    model_id = top_1['Model']
    selectors = re.findall(r'\d\d', top_1['Parameters'])
    dtc = DecisionTreeClassifier(max_depth=int(selectors[0]), min_samples_split=int(selectors[1]), max_features=int(selectors[2]),\
                                            random_state=514)
    
    dtc.fit(X_train[features], y_train)

    comparison_chart = model.create_comp_chart()
    comparison_chart[model_id] = model.compute_metrics(dtc, X_test[features], y_test).values

    score = dtc.score(X_test[features], y_test).round(4)

    description = pd.DataFrame({'Model': model_id,
                                'Accuracy(Score)': score,
                                'Type': 'Decision Tree Classifier',
                                'Features Used': f'{feat_set}',
                                'Parameters': f'Depth: {selectors[0]}'},
                                index=[0])

    return description, comparison_chart

In [136]:
test_description, test_chart = test_dtc(feat_set, top_1, subsets)

In [137]:
test_chart

,DTC_90
Accuracy/Score,0.9463
True Positives,9372.0000
False Positives,289.0000
True Negatives,29.0000
False Negatives,245.0000
TPR/Recall,0.9745
False Positive Rate,0.9088
True Negative Rate,0.0912
False Negative Rate,0.0255
Precision,0.9701


## Takeaways 
* Even worse performance on with the True Negative rate. Attempting to control for overfitting caused a very strong hit. Will have to readjust next go through. 

# Modeling With Linear SVC